In [6]:
import os
from __future__ import division

import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable
import numpy as np
import torch.optim as optim

cfgfile = "C:/Users/HX/Desktop/yolov4.cfg"
model_file_path = "Model/model.pt"

def read_cfg_file(cfgfile):
    file = open(cfgfile, 'r')
    lines = file.read().split('\n')

    layer_type = []
    layer_details = []
    current_layer_details = {}
    for line in lines:
        #print(line)
        if line == '':
            continue
        elif line[0] == '#':
            continue
        else:
            if (line[0] == '['):
                layer_type.append(line[1 : -1])
                if current_layer_details != {}:
                    layer_details.append(current_layer_details)
                    current_layer_details = {}
            else:
                current_layer_details.update([(line.split("=")[0].rstrip(), line.split("=")[1].lstrip())])
    layer_details.append(current_layer_details)
    return layer_type, layer_details

 

In [7]:
class Mish(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x = x * torch.tanh(F.softplus(x))
        return x

class Conv_Layer_box(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size, stride, activation_func, batch_normalization):
        super().__init__()
        padding = (int((kernel_size - 1)/2), int((kernel_size - 1)/2))
        #TBC: linear
        dict_activation_func = {"ReLU": nn.ReLU(inplace=False),
                                "linear": nn.ReLU(inplace=False),
                                "leaky": nn.LeakyReLU(0.1, inplace=False),
                                "mish": Mish()
                               }
        
        if batch_normalization == True:
            bias = False
        else:
            bias = True
        self.conv_box = nn.ModuleList()
        self.conv_box.append(nn.Conv2d(in_channel, out_channel, kernel_size, stride, padding, bias = bias))
        if batch_normalization == True:
            self.conv_box.append(nn.BatchNorm2d(out_channel))
        self.conv_box.append(dict_activation_func[activation_func])
        
    def forward(self, x):
        for layer in self.conv_box:
            x = layer(x)
        return x
    
class Maxpool_pad_Layer_box(nn.Module):
    def __init__(self, maxpool_size):
        super().__init__()
        self.maxpool_size = maxpool_size
        #why there are 2 padding??????????????
        self.pad_1 = int((self.maxpool_size - 1) / 2)
        self.pad_2 = self.pad_1
    def forward(self, x):
        x = F.pad(x, (self.pad_1, self.pad_2, self.pad_1, self.pad_2), mode='replicate')
        x = F.max_pool2d(x, self.maxpool_size, stride=1)
        return x
    
class Upsample_layer(nn.Module):
    def __init__(self, stride):
        super().__init__()
        self.stride = stride
        
    def forward(self, x):
        batch, channel, height, width = x.data.size()
        x = x.view(batch, channel, height, 1, width, 1).expand(batch, channel, height, self.stride, width, self.stride)
        x = x.contiguous().view(batch, channel, height * self.stride, width * self.stride)
        return x
    
class Yolo(nn.Module):
    def __init__(self):
        super().__init__()
        
        
class shortcut(nn.Module):
    def __init__(self):
        super().__init__()
        
class route(nn.Module):
    def __init__(self):
        super().__init__()

In [8]:
layer_type, layer_details = read_cfg_file(cfgfile)

net_layer = layer_details[0]

layer_type = layer_type[1:]
layer_details = layer_details[1:]

print(len(layer_type))
print(len(layer_details))
print(layer_details[1])

162
162
{'batch_normalize': '1', 'filters': '64', 'size': '3', 'stride': '2', 'pad': '1', 'activation': 'mish'}


In [9]:
#build module for entire YOLO
class YOLO_v4_model(nn.Module):
    def __init__(self, layer_details, layer_type):
        super(YOLO_v4_model, self).__init__()
        self.all_layers = nn.ModuleList()
        all_layers = self.all_layers
        self.layer_details = layer_details
        self.layer_type = layer_type

        for i in range(len(layer_type)):
            if layer_type[i] == 'convolutional':
                #print(layer_details[i])
                #print(i)
                try:
                    if int(layer_details[i]['batch_normalize']) == 1:
                        batch_normalize = True
                    else:
                        batch_normalize = False
                except:
                    batch_normalize = False
                if i == 0:
                    in_channel = 3
                else:
                    if layer_type[i - 1] == 'convolutional':
                        skip_step = 0
                    elif layer_type[i - 1] == 'shortcut':
                        skip_step = int(layer_details[i - 1]['from'])
                    elif layer_type[i - 1] == 'route':
                        try:
                            skip_step = int(layer_details[i - 1]['layers'].split(",")[0])
                        except:
                            skip_step = int(layer_details[i - 1]['layers'])
                    #print(layer_details[i - 1 + skip_step])
                    if skip_step > 0:
                        in_channel = int(layer_details[skip_step]['filters'])
                    else:
                        in_channel = int(layer_details[i - 1 + skip_step]['filters'])
                out_channel = int(layer_details[i]['filters'])
                kernel_size = int(layer_details[i]['size'])
                stride = int(layer_details[i]['stride'])
                pad = int(layer_details[i]['pad'])
                activation_func = layer_details[i]['activation']
                layer = Conv_Layer_box(in_channel, out_channel, kernel_size, stride, activation_func, batch_normalize)
                #print(layer)
            elif layer_type[i] == 'maxpool':
                layer_details[i].update([('filters', layer_details[i - 1]['filters'])])
                maxpool_size = int(layer_details[i]['size'])
                #print(maxpool_size)
                layer = Maxpool_pad_Layer_box(maxpool_size)
                #print(layer)
            elif layer_type[i] == 'upsample':
                layer_details[i].update([('filters', layer_details[i - 1]['filters'])])
                stride = int(layer_details[i]['stride'])
                layer = Upsample_layer(stride)
            elif layer_type[i] == 'yolo':
                #print("yolo")
                layer = Yolo()
                #continue
            elif layer_type[i] == 'shortcut':
                skip_step = int(layer_details[i]['from'])
                layer_details[i].update([('filters', layer_details[i + skip_step]['filters'])])
                layer = shortcut()
            elif layer_type[i] == 'route':
                try:
                    skip_step = int(layer_details[i]['layers'].split(",")[0])
                except:
                    skip_step = int(layer_details[i]['layers'])
                #print(skip_step)
                if skip_step > 0:
                    layer_details[i].update([('filters', layer_details[skip_step]['filters'])])
                else:
                    layer_details[i].update([('filters', layer_details[i + skip_step]['filters'])])
                layer = route()
            elif layer_type[i] == 'net':
                #print("net")
                continue
            else:
                continue

            all_layers.append(layer)
        global all_layerrr
        all_layerrr = all_layers

    def forward(self, x):
        all_layers = self.all_layers
        layers_output = [None for i in range(len(layer_type))]
        for i in range(len(layer_type)):
            #print(i)
            if i == 0:
                layers_output[i] = all_layers[i](x)
                continue
                
            elif layer_type[i] == 'yolo':
                continue
            elif layer_type[i] == 'convolutional' or layer_type[i] == 'maxpool' or layer_type[i] == 'upsample' or layer_type[i] == 'yolo':
                layers_output[i] = all_layers[i](layers_output[i - 1])
                """
                try:
                    print("i: " + str(i) + str(layers_output[i].size()))
                except:
                    print("go")
                """
                continue
            elif layer_type[i] == 'shortcut':
                skip_step = [int(layer_details[i]['from'])]
            elif layer_type[i] == 'route':
                skip_step = layer_details[i]['layers'].split(",")
            for SS in skip_step:
                SS = int(SS)
                #print("SS" + str(i) + str(SS))
                #print(skip_step)
                
                #print(SS)
                #print(layers_output[i])
                #print(layers_output[i - 1 + SS])
                
                if SS > 0:
                    if layers_output[i] == None:
                        #print(layers_output[SS].size())
                        layers_output[i] = layers_output[SS]
                    else:
                        #print(layers_output[SS].size())
                        layers_output[i] += layers_output[SS]
                else:
                    #print(i + SS)
                    if layers_output[i] == None:
                        #print(layers_output[i + SS].size())
                        layers_output[i] = layers_output[i + SS]
                    else:
                        
                        #print(layers_output[i + SS].size())
                        layers_output[i] += layers_output[i + SS]
            """
            try:
                print("i: " + str(i) + str(layers_output[i].size()))
            except:
                print("go")
            """
        return layers_output[160]
    



In [10]:
input = np.array([1 for i in range(608 * 608 * 3)]).reshape(1, 3, 608, 608)
#target = np.array([0 for i in range(7 * 7 * 30)])
target = np.array([0 for i in range(1 * 255 * 19 * 19)])

input_tensor = torch.Tensor(input)
output_tensor = torch.Tensor(target)

#x = np.array([1 for i in range(608 * 608 * 3)]).reshape(1, 3, 608, 608)
#x = torch.tensor(x)

learning_rate = 0.08
epoch_size = 5
steps_for_printing_out_loss = 1

YOLO_v4_Module_WIP = YOLO_v4_model(layer_details, layer_type)
YOLO_v4_Module_WIP.cuda()
#Model_WIP.to(device)
loss_functioin = nn.MSELoss()
optimizer = optim.SGD(YOLO_v4_Module_WIP.parameters(), lr = learning_rate)

input = input_tensor.cuda()
target = output_tensor.cuda()

def training_model():
    for i in range(1, epoch_size + 1):
        optimizer.zero_grad()
        output = YOLO_v4_Module_WIP(input.cuda())
        print(output.size())
        loss = loss_functioin(output, target.reshape(output.size(0), output.size(1), output.size(2), output.size(3)))
        loss.backward()
        optimizer.step()
        if i % (steps_for_printing_out_loss) == 0:
            print('Loss (epoch: ' + str(i) + '): ' + str(loss.cpu().detach().numpy()))
    torch.save({'state_dict': YOLO_v4_Module_WIP.state_dict(),'optimizer': optimizer.state_dict()}, model_file_path)

training_model()

torch.Size([1, 255, 19, 19])
Loss (epoch: 1): 0.08263133
torch.Size([1, 255, 19, 19])
Loss (epoch: 2): 0.07485464
torch.Size([1, 255, 19, 19])
Loss (epoch: 3): 0.069151476
torch.Size([1, 255, 19, 19])
Loss (epoch: 4): 0.06465164
torch.Size([1, 255, 19, 19])
Loss (epoch: 5): 0.060577523


In [11]:
all_layerrr

ModuleList(
  (0): Conv_Layer_box(
    (conv_box): ModuleList(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Mish()
    )
  )
  (1): Conv_Layer_box(
    (conv_box): ModuleList(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Mish()
    )
  )
  (2): Conv_Layer_box(
    (conv_box): ModuleList(
      (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Mish()
    )
  )
  (3): route()
  (4): Conv_Layer_box(
    (conv_box): ModuleList(
      (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True

In [12]:
layer_details[160]

{'size': '1',
 'stride': '1',
 'pad': '1',
 'filters': '255',
 'activation': 'linear'}

In [13]:
#all_layers
output

NameError: name 'output' is not defined

In [ ]:
class YOLO_v4_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.YOLO_v4_layers = nn.ModuleList()
        
            self.YOLO_v4_layers.append(Conv_Layer_box(in_channel[i], out_channel[i], kernel_size= kernel_size[i], stride = stride[i], activation_func = activation_func[i], batch_normalization = batch_normalization[i]))
    

In [ ]:
class abc():
    def __init__(self, qwe, out):
        print(qwe)
        
abc(4,5)

In [ ]:
abc = "1,2,3"
m = abc.split(",")
m
abc = "1"
m = abc.split(",")
m
k = 4

for r in k:
    print(r)